In [0]:
%spark2.pyspark

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace, col, when
from pyspark.sql.types import IntegerType

spark = SparkSession.builder.appName("SimanETL").getOrCreate()

simanRawData = spark.read.csv("hdfs:///user/hdfs/simanData/rawData/simanRawData.csv", header=True)

In [1]:
%spark2.pyspark

simanRawData = simanRawData.na.fill(value="0",subset=["Price1"])

simanRawData = simanRawData.na.fill(value="0",subset=["Price2"])

simanRawData = simanRawData.na.fill(value="0",subset=["Discount"])

simanRawData = simanRawData.na.fill(value="Sin imagen",subset=["Image"])

simanRawData = simanRawData.na.fill(value="Sin nombre",subset=["Name"])

simanRawData = simanRawData.na.fill(value="Sin marca",subset=["Brand"])

simanRawData = simanRawData.na.fill(value="Sin link",subset=["Link"])

simanRawData.show()

In [2]:
%spark2.pyspark

simanRawData = simanRawData.withColumn('Price1', regexp_replace('Price1', 'Q&nbsp;', '')).withColumn('Price1', regexp_replace('Price1', ',', ''))

simanRawData = simanRawData.withColumn('Price2', regexp_replace('Price2', 'Q&nbsp;', '')).withColumn('Price2', regexp_replace('Price2', ',', ''))

simanRawData = simanRawData.withColumn('Discount', regexp_replace('Discount', '&nbsp;%', ''))

simanRawData.show()


In [3]:
%spark2.pyspark

simanRawData = simanRawData.withColumn('Price1', col('Price1').cast(IntegerType()))

simanRawData = simanRawData.withColumn('Price2', col('Price2').cast(IntegerType()))

simanRawData = simanRawData.withColumn('Discount', col('Discount').cast(IntegerType()))

simanRawData.printSchema()





In [4]:
%spark2.pyspark

simanRawData = simanRawData.withColumn('Price', when(simanRawData.Price1 >= simanRawData.Price2, simanRawData.Price1).otherwise(simanRawData.Price2))

simanRawData = simanRawData.withColumn('PriceWithDiscount', when(simanRawData.Price1 >= simanRawData.Price2, simanRawData.Price2).otherwise(simanRawData.Price1))

simanRawData = simanRawData.drop('Price1')

simanRawData = simanRawData.drop('Price2')

simanRawData.show()

In [5]:
%spark2.pyspark

simanRawData.write.save('/user/hdfs/simanData/cleanData', format='csv', mode='append')